In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[0-160],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[160-260],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[160-260],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


In [3]:

terms = ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas']
rate_plans = ['Standard', 'Nonref']
weekend_stays = ['True', 'False']
length_of_stay_buckets = ['[0-1]', '[2-3]', '[4-7]', '[8-inf]']
room_segments = ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]']
n_people_buckets = ['[1-1]', '[2-2]', '[3-4]', '[5-inf]']

length_of_stay_bucket_to_min   = { '[0-1]':0,  '[2-3]':2,  '[4-7]':4,  '[8-inf]':8,  np.nan:0 }
length_of_stay_bucket_to_value = { '[0-1]':1.5,'[2-3]':2.5,'[4-7]':5.5,'[8-inf]':15, np.nan:0 }
length_of_stay_bucket_to_max   = { '[0-1]':1,  '[2-3]':3,  '[4-7]':7,  '[8-inf]':20, np.nan:0 }

room_segment_to_min   = { '[0-160]':0,  '[160-260]':160,'[260-360]':260,'[360-500]':360,'[500-900]':500, np.nan:0 }
room_segment_to_value = { '[0-160]':80, '[160-260]':210,'[260-360]':310,'[360-500]':430,'[500-900]':700, np.nan:0 }
room_segment_to_max   = { '[0-160]':160,'[160-260]':260,'[260-360]':360,'[360-500]':500,'[500-900]':900, np.nan:0 }

n_people_bucket_to_min   = { '[1-1]':1,'[2-2]':2,'[3-4]':3,  '[5-inf]':5,  np.nan:0 }
n_people_bucket_to_value = { '[1-1]':1,'[2-2]':2,'[3-4]':3.5,'[5-inf]':7,  np.nan:0 }
n_people_bucket_to_max   = { '[1-1]':1,'[2-2]':2,'[3-4]':4,  '[5-inf]':10, np.nan:0 }


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [4]:

def user_add_n_inter(add_features,interactions_df,users_df,user_features):
    
    inter_counts = interactions_df.groupby("user_id").count().reset_index()[["user_id","item_id"]]
    inter_counts = inter_counts.rename(columns={"item_id":"user:n_inter"})
    
    users_df = users_df.merge(inter_counts,on="user_id")
    
    if add_features:
        user_features.append("user:n_inter")
    return users_df,user_features

def user_add_one_hot_values(add_features,interactions_df,users_df,user_features):
    
    inter_df = interactions_df.copy()
    inter_user_ids = inter_df["user_id"]
    
    one_hot_term = pd.get_dummies(inter_df["term"])
    one_hot_term = one_hot_term.add_prefix("user:one_hot:term:")
    
    one_hot_plan = pd.get_dummies(inter_df["rate_plan"])
    one_hot_plan = one_hot_plan.add_prefix("user:one_hot:rate_plan:")
    
    one_hot_weekend = pd.get_dummies(inter_df["weekend_stay"])
    one_hot_weekend = one_hot_weekend.add_prefix("user:one_hot:weekend_stay:")
    
    one_hot_stay = pd.get_dummies(inter_df["length_of_stay_bucket"])
    one_hot_stay = one_hot_stay.add_prefix("user:one_hot:length_of_stay_bucket:")
    
    one_hot_room = pd.get_dummies(inter_df["room_segment"])
    one_hot_room = one_hot_room.add_prefix("user:one_hot:room_segment:")
    
    one_hot_n_people = pd.get_dummies(inter_df["n_people_bucket"])
    one_hot_n_people = one_hot_n_people.add_prefix("user:one_hot:n_people_bucket:")
    
    one_hot = one_hot_term.copy()
    one_hot = one_hot.join(one_hot_plan)
    one_hot = one_hot.join(one_hot_weekend)
    one_hot = one_hot.join(one_hot_stay)
    one_hot = one_hot.join(one_hot_room)
    one_hot = one_hot.join(one_hot_n_people)
    one_hot = one_hot.join(inter_user_ids)
    one_hot = one_hot.groupby("user_id").sum()
    
    for index,row in one_hot.iterrows():
        for column in one_hot.columns:
            row[column] = 1 if row[column]>0 else 0
    
    if add_features:
        user_features.extend(one_hot.columns.tolist())
    
    one_hot = one_hot.reset_index()
    users_df = users_df.merge(one_hot,on="user_id")
    
    return users_df,user_features

def user_add_value_counts(add_features,interactions_df,users_df,user_features,user_id_mapping):
    
    user_terms = interactions_df[["user_id","term"]].value_counts()
    for term in terms:
        users_df["user:n:term:"+term] = 0
        if add_features:
            user_features.append("user:n:term:"+term)
    for (user,term),value in user_terms.iteritems():
        users_df.loc[user_id_mapping[user],"user:n:term:"+term] = value
    
    user_plans = interactions_df[["user_id","rate_plan"]].value_counts()
    for plan in rate_plans:
        users_df["user:n:rate_plan:"+plan] = 0
        if add_features:
            user_features.append("user:n:rate_plan:"+plan)
    for (user,plan),value in user_plans.iteritems():
        users_df.loc[user_id_mapping[user],"user:n:rate_plan:"+plan] = value
    
    user_weekends = interactions_df[["user_id","weekend_stay"]].value_counts()
    for weekend in weekend_stays:
        users_df["user:n:weekend_stay:"+weekend] = 0
        if add_features:
            user_features.append("user:n:weekend_stay:"+weekend)
    for (user,weekend),value in user_weekends.iteritems():
        users_df.loc[user_id_mapping[user],"user:n:weekend_stay:"+weekend] = value
    
    user_stays = interactions_df[["user_id","length_of_stay_bucket"]].value_counts()
    for stay in length_of_stay_buckets:
        users_df["user:n:length_of_stay_bucket:"+stay] = 0
        if add_features:
            user_features.append("user:n:length_of_stay_bucket:"+stay)
    for (user,stay),value in user_stays.iteritems():
        users_df.loc[user_id_mapping[user],"user:n:length_of_stay_bucket:"+stay] = value
    
    user_rooms = interactions_df[["user_id","room_segment"]].value_counts()
    for room in room_segments:
        users_df["user:n:room_segment:"+room] = 0
        if add_features:
            user_features.append("user:n:room_segment:"+room)
    for (user,room),value in user_rooms.iteritems():
        users_df.loc[user_id_mapping[user],"user:n:room_segment:"+room] = value
    
    user_n_people = interactions_df[["user_id","n_people_bucket"]].value_counts()
    for n_people in n_people_buckets:
        users_df["user:n:n_people_bucket:"+n_people] = 0
        if add_features:
            user_features.append("user:n:n_people_bucket:"+n_people)
    for (user,n_people),value in user_n_people.iteritems():
        users_df.loc[user_id_mapping[user],"user:n:n_people_bucket:"+n_people] = value
    
    return users_df,user_features

def user_add_value_probs(add_features,interactions_df,users_df,user_features):
    
    for term in terms:
        users_df["user:prob:term:"+term] = \
            users_df["user:n:term:"+term]/users_df["user:n_inter"]
        if add_features:
            user_features.append("user:prob:term:"+term)
    
    for plan in rate_plans:
        users_df["user:prob:rate_plan:"+plan] = \
            users_df["user:n:rate_plan:"+plan]/users_df["user:n_inter"]
        if add_features:
            user_features.append("user:prob:rate_plan:"+plan)
    
    for weekend in weekend_stays:
        users_df["user:prob:weekend_stay:"+weekend] = \
            users_df["user:n:weekend_stay:"+weekend]/users_df["user:n_inter"]
        if add_features:
            user_features.append("user:prob:weekend_stay:"+weekend)
    
    for stay in length_of_stay_buckets:
        users_df["user:prob:length_of_stay_bucket:"+stay] = \
            users_df["user:n:length_of_stay_bucket:"+stay]/users_df["user:n_inter"]
        if add_features:
            user_features.append("user:prob:length_of_stay_bucket:"+stay)
    
    for room in room_segments:
        users_df["user:prob:room_segment:"+room] = \
            users_df["user:n:room_segment:"+room]/users_df["user:n_inter"]
        if add_features:
            user_features.append("user:prob:room_segment:"+room)
    
    for n_people in n_people_buckets:
        users_df["user:prob:n_people_bucket:"+n_people] = \
            users_df["user:n:n_people_bucket:"+n_people]/users_df["user:n_inter"]
        if add_features:
            user_features.append("user:prob:n_people_bucket:"+n_people)
    
    return users_df,user_features

def user_add_avg_values(add_features,interactions_df,users_df,user_features):
    
    stay_sum = 0
    for stay in length_of_stay_buckets:
        stay_sum += length_of_stay_bucket_to_value[stay]*users_df["user:n:length_of_stay_bucket:"+stay]
    users_df["user:avg_length_of_stay_bucket"] = stay_sum/users_df["user:n_inter"]
    if add_features:
        user_features.append("user:avg_length_of_stay_bucket")
    
    room_sum = 0
    for room in room_segments:
        room_sum += room_segment_to_value[room]*users_df["user:n:room_segment:"+room]
    users_df["user:avg_room_segment"] = room_sum/users_df["user:n_inter"]
    if add_features:
        user_features.append("user:avg_room_segment")
    
    n_people_sum = 0
    for n_people in n_people_buckets:
        n_people_sum += n_people_bucket_to_value[n_people]*users_df["user:n:n_people_bucket:"+n_people]
    users_df["user:avg_n_people_bucket"] = n_people_sum/users_df["user:n_inter"]
    if add_features:
        user_features.append("user:avg_n_people_bucket")
    
    return users_df,user_features

def user_add_pop_values(add_features,interactions_df,users_df,user_features):
    
    def get_most_pup_term(user_row):
        term_dict={}
        for term in terms:
            term_dict[term] = user_row["user:n:term:"+term]
        return terms.index(max(term_dict,key=term_dict.get))
    
    def get_most_pup_rate_plan(user_row):
        plan_dict={}
        for plan in rate_plans:
            plan_dict[plan] = user_row["user:n:rate_plan:"+plan]
        return rate_plans.index(max(plan_dict,key=plan_dict.get))
    
    def get_most_pup_weekend_stay(user_row):
        weekend_dict={}
        for weekend in weekend_stays:
            weekend_dict[weekend] = user_row["user:n:weekend_stay:"+weekend]
        return weekend_stays.index(max(weekend_dict,key=weekend_dict.get))
    
    def get_most_pup_length_of_stay_bucket(user_row):
        stay_dict={}
        for stay in length_of_stay_buckets:
            stay_dict[stay] = user_row["user:n:length_of_stay_bucket:"+stay]
        return length_of_stay_buckets.index(max(stay_dict,key=stay_dict.get))
    
    def get_most_pup_room_segment(user_row):
        room_dict={}
        for room in room_segments:
            room_dict[room] = user_row["user:n:room_segment:"+room]
        return room_segments.index(max(room_dict,key=room_dict.get))
    
    def get_most_pup_n_people_bucket(user_row):
        n_people_dict={}
        for n_people in n_people_buckets:
            n_people_dict[n_people] = user_row["user:n:n_people_bucket:"+n_people]
        return n_people_buckets.index(max(n_people_dict,key=n_people_dict.get))
    
    users_df["user:most_pop_term"] = users_df.apply(get_most_pup_term,axis=1)
    if add_features:
        user_features.append("user:most_pop_term")
    users_df["user:most_pop_rate_plan"] = users_df.apply(get_most_pup_rate_plan,axis=1)
    if add_features:
        user_features.append("user:most_pop_rate_plan")
    users_df["user:most_pop_weekend_stay"] = users_df.apply(get_most_pup_weekend_stay,axis=1)
    if add_features:
        user_features.append("user:most_pop_weekend_stay")
    users_df["user:most_pop_length_of_stay_bucket"] = users_df.apply(get_most_pup_length_of_stay_bucket,axis=1)
    if add_features:
        user_features.append("user:most_pop_length_of_stay_bucket")
    users_df["user:most_pop_room_segment"] = users_df.apply(get_most_pup_room_segment,axis=1)
    if add_features:
        user_features.append("user:most_pop_room_segment")
    users_df["user:most_pop_n_people_bucket"] = users_df.apply(get_most_pup_n_people_bucket,axis=1)
    if add_features:
        user_features.append("user:most_pop_n_people_bucket")
    
    return users_df,user_features

def prepare_users_df(interactions_df):
    
    user_ids = interactions_df["user_id"].unique()
    n_users = len(user_ids)
    user_id_mapping = dict(zip(user_ids, list(range(n_users))))
    user_id_reverse_mapping = dict(zip(list(range(n_users)), user_ids))
    user_features = []
    users_df = pd.DataFrame({"user_id":user_ids})
    
#     users_df,user_features = user_add_n_inter        (True,interactions_df,users_df,user_features)
    users_df,user_features = user_add_one_hot_values (True,interactions_df,users_df,user_features)
#     users_df,user_features = user_add_value_counts   (False,interactions_df,users_df,user_features,user_id_mapping)
#     users_df,user_features = user_add_value_probs    (True,interactions_df,users_df,user_features)
#     users_df,user_features = user_add_avg_values     (True,interactions_df,users_df,user_features)
#     users_df,user_features = user_add_pop_values     (True,interactions_df,users_df,user_features)
    
    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))


['user:one_hot:term:WinterVacation', 'user:one_hot:term:Easter', 'user:one_hot:term:OffSeason', 'user:one_hot:term:HighSeason', 'user:one_hot:term:LowSeason', 'user:one_hot:term:MayLongWeekend', 'user:one_hot:term:NewYear', 'user:one_hot:term:Christmas', 'user:one_hot:rate_plan:Standard', 'user:one_hot:rate_plan:Nonref', 'user:one_hot:weekend_stay:True', 'user:one_hot:weekend_stay:False', 'user:one_hot:length_of_stay_bucket:[0-1]', 'user:one_hot:length_of_stay_bucket:[2-3]', 'user:one_hot:length_of_stay_bucket:[4-7]', 'user:one_hot:length_of_stay_bucket:[8-inf]', 'user:one_hot:room_segment:[0-160]', 'user:one_hot:room_segment:[160-260]', 'user:one_hot:room_segment:[260-360]', 'user:one_hot:room_segment:[360-500]', 'user:one_hot:room_segment:[500-900]', 'user:one_hot:n_people_bucket:[1-1]', 'user:one_hot:n_people_bucket:[2-2]', 'user:one_hot:n_people_bucket:[3-4]', 'user:one_hot:n_people_bucket:[5-inf]']


,user_id,user:one_hot:term:WinterVacation,user:one_hot:term:Easter,user:one_hot:term:OffSeason,user:one_hot:term:HighSeason,user:one_hot:term:LowSeason,user:one_hot:term:MayLongWeekend,user:one_hot:term:NewYear,user:one_hot:term:Christmas,user:one_hot:rate_plan:Standard,...,user:one_hot:length_of_stay_bucket:[8-inf],user:one_hot:room_segment:[0-160],user:one_hot:room_segment:[160-260],user:one_hot:room_segment:[260-360],user:one_hot:room_segment:[360-500],user:one_hot:room_segment:[500-900],user:one_hot:n_people_bucket:[1-1],user:one_hot:n_people_bucket:[2-2],user:one_hot:n_people_bucket:[3-4],user:one_hot:n_people_bucket:[5-inf]
0,1,1,0,1,1,1,0,0,0,1,...,0,1,1,1,1,1,0,1,1,1
40,50,1,0,1,1,1,0,0,0,1,...,0,1,1,1,1,0,0,1,1,1
321,706,1,0,1,1,1,1,1,0,1,...,1,1,1,1,1,0,1,1,1,1
547,115,0,0,1,0,0,0,0,0,1,...,0,1,1,0,0,0,1,0,0,1
1270,1736,1,0,1,1,1,0,0,0,1,...,0,1,1,1,1,0,1,1,1,0
2785,96,1,0,1,1,1,1,0,0,1,...,1,1,1,1,1,0,1,1,1,1
11527,7779,0,0,1,0,1,0,0,0,1,...,1,1,1,1,0,0,0,0,1,1


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [5]:

def item_add_one_hot_values(add_features,interactions_df,items_df,item_features):
    
    uniq_inter_df = interactions_df.copy()
    uniq_inter_df = uniq_inter_df.drop(columns=["user_id"])
    uniq_inter_df = uniq_inter_df.drop_duplicates()
    uniq_inter_df = uniq_inter_df.reset_index()
    
    one_hot_term = pd.get_dummies(uniq_inter_df["term"])
    one_hot_term = one_hot_term.add_prefix("item:one_hot:term:")
    
    one_hot_plan = pd.get_dummies(uniq_inter_df["rate_plan"])
    one_hot_plan = one_hot_plan.add_prefix("item:one_hot:rate_plan:")
    
    one_hot_weekend = pd.get_dummies(uniq_inter_df["weekend_stay"])
    one_hot_weekend = one_hot_weekend.add_prefix("item:one_hot:weekend_stay:")
    
    one_hot_stay = pd.get_dummies(uniq_inter_df["length_of_stay_bucket"])
    one_hot_stay = one_hot_stay.add_prefix("item:one_hot:length_of_stay_bucket:")
    
    one_hot_room = pd.get_dummies(uniq_inter_df["room_segment"])
    one_hot_room = one_hot_room.add_prefix("item:one_hot:room_segment:")
    
    one_hot_n_people = pd.get_dummies(uniq_inter_df["n_people_bucket"])
    one_hot_n_people = one_hot_n_people.add_prefix("item:one_hot:n_people_bucket:")
    
    one_hot = one_hot_term.copy()
    one_hot = one_hot.join(one_hot_plan)
    one_hot = one_hot.join(one_hot_weekend)
    one_hot = one_hot.join(one_hot_stay)
    one_hot = one_hot.join(one_hot_room)
    one_hot = one_hot.join(one_hot_n_people)
    
    items_df = items_df.join(one_hot)
    
    if add_features:
        item_features.extend(one_hot.columns.tolist())
    
    return items_df,item_features

def item_add_min_of_bucket(add_features,interactions_df,items_df,item_features):
    
    def get_min_length_of_stay_bucket(item_row):
        stays = []
        for stay in length_of_stay_buckets:
            if item_row["item:one_hot:length_of_stay_bucket:"+stay]==1:
                stays.append(length_of_stay_bucket_to_value[stay])
        return 0 if len(stays)==0 else np.min(stays)
    
    def get_min_room_segment(item_row):
        rooms = []
        for room in room_segments:
            if item_row["item:one_hot:room_segment:"+room]==1:
                rooms.append(room_segment_to_value[room])
        return 0 if len(rooms)==0 else np.min(rooms)
        
    def get_min_n_people_bucket(item_row):
        n_peoples = []
        for n_people in n_people_buckets:
            if item_row["item:one_hot:n_people_bucket:"+n_people]==1:
                n_peoples.append(n_people_bucket_to_value[n_people])
        return 0 if len(n_peoples)==0 else np.min(n_peoples)
    
    items_df["item:min_length_of_stay_bucket"] = items_df.apply(get_min_length_of_stay_bucket,axis=1)
    if add_features:
        item_features.append("item:min_length_of_stay_bucket")
    items_df["item:min_room_segment"] = items_df.apply(get_min_room_segment,axis=1)
    if add_features:
        item_features.append("item:min_room_segment")
    items_df["item:min_n_people_bucket"] = items_df.apply(get_min_n_people_bucket,axis=1)
    if add_features:
        item_features.append("item:min_n_people_bucket")
    
    return items_df,item_features

def item_add_max_of_bucket(add_features,interactions_df,items_df,item_features):
    
    def get_max_length_of_stay_bucket(item_row):
        stays = []
        for stay in length_of_stay_buckets:
            if item_row["item:one_hot:length_of_stay_bucket:"+stay]==1:
                stays.append(length_of_stay_bucket_to_value[stay])
        return 0 if len(stays)==0 else np.max(stays)
    
    def get_max_room_segment(item_row):
        rooms = []
        for room in room_segments:
            if item_row["item:one_hot:room_segment:"+room]==1:
                rooms.append(room_segment_to_value[room])
        return 0 if len(rooms)==0 else np.max(rooms)
        
    def get_max_n_people_bucket(item_row):
        n_peoples = []
        for n_people in n_people_buckets:
            if item_row["item:one_hot:n_people_bucket:"+n_people]==1:
                n_peoples.append(n_people_bucket_to_value[n_people])
        return 0 if len(n_peoples)==0 else np.max(n_peoples)
    
    items_df["item:max_length_of_stay_bucket"] = items_df.apply(get_max_length_of_stay_bucket,axis=1)
    if add_features:
        item_features.append("item:max_length_of_stay_bucket")
    items_df["item:max_room_segment"] = items_df.apply(get_max_room_segment,axis=1)
    if add_features:
        item_features.append("item:max_room_segment")
    items_df["item:max_n_people_bucket"] = items_df.apply(get_max_n_people_bucket,axis=1)
    if add_features:
        item_features.append("item:max_n_people_bucket")
    
    return items_df,item_features

def prepare_items_df(interactions_df):
    
    item_ids = interactions_df['item_id'].unique()
    n_items = len(item_ids)
    items_df = pd.DataFrame({"item_id":item_ids})
    item_features = []
    
    items_df,item_features = item_add_one_hot_values (True,interactions_df,items_df,item_features)
#     items_df,item_features = item_add_min_of_bucket  (True,interactions_df,items_df,item_features)
#     items_df,item_features = item_add_max_of_bucket  (True,interactions_df,items_df,item_features)
    
    return items_df,item_features

items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))


['item:one_hot:term:WinterVacation', 'item:one_hot:term:Easter', 'item:one_hot:term:OffSeason', 'item:one_hot:term:HighSeason', 'item:one_hot:term:LowSeason', 'item:one_hot:term:MayLongWeekend', 'item:one_hot:term:NewYear', 'item:one_hot:term:Christmas', 'item:one_hot:rate_plan:Standard', 'item:one_hot:rate_plan:Nonref', 'item:one_hot:weekend_stay:True', 'item:one_hot:weekend_stay:False', 'item:one_hot:length_of_stay_bucket:[0-1]', 'item:one_hot:length_of_stay_bucket:[2-3]', 'item:one_hot:length_of_stay_bucket:[4-7]', 'item:one_hot:length_of_stay_bucket:[8-inf]', 'item:one_hot:room_segment:[0-160]', 'item:one_hot:room_segment:[160-260]', 'item:one_hot:room_segment:[260-360]', 'item:one_hot:room_segment:[360-500]', 'item:one_hot:room_segment:[500-900]', 'item:one_hot:n_people_bucket:[1-1]', 'item:one_hot:n_people_bucket:[2-2]', 'item:one_hot:n_people_bucket:[3-4]', 'item:one_hot:n_people_bucket:[5-inf]']


,item_id,item:one_hot:term:WinterVacation,item:one_hot:term:Easter,item:one_hot:term:OffSeason,item:one_hot:term:HighSeason,item:one_hot:term:LowSeason,item:one_hot:term:MayLongWeekend,item:one_hot:term:NewYear,item:one_hot:term:Christmas,item:one_hot:rate_plan:Standard,...,item:one_hot:length_of_stay_bucket:[8-inf],item:one_hot:room_segment:[0-160],item:one_hot:room_segment:[160-260],item:one_hot:room_segment:[260-360],item:one_hot:room_segment:[360-500],item:one_hot:room_segment:[500-900],item:one_hot:n_people_bucket:[1-1],item:one_hot:n_people_bucket:[2-2],item:one_hot:n_people_bucket:[3-4],item:one_hot:n_people_bucket:[5-inf]
0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
2,2,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
3,3,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,4,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
5,5,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
6,6,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        negative_interactions = []
        
        ########################
    
        n_users = np.max(interactions_df['user_id'])+1
        n_items = np.max(interactions_df['item_id'])+1
        n_interactions = len(interactions_df)
        
        r = np.zeros(shape=(n_users,n_items))
        for idx,interaction in interactions_df.iterrows():
            r[int(interaction['user_id'])][int(interaction['item_id'])] = 1
    
        i = 0
        while i < self.n_neg_per_pos*n_interactions:
            sample_size = 1000
            user_ids = self.rng.choice(np.arange(n_users),size=sample_size)
            item_ids = self.rng.choice(np.arange(n_items),size=sample_size)
            j = 0
            while j < sample_size and i < self.n_neg_per_pos*n_interactions:
                if r[user_ids[j]][item_ids[j]] == 0:
                    negative_interactions.append([user_ids[j],item_ids[j],0])
                    i += 1
                j += 1
        
        ########################
        # Write your code here #
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions 
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.
        
        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values
    
        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        ########################
        users_df = users_df.merge(self.users_df,on="user_id",how="left").fillna(0)
        items_df, item_features = prepare_items_df(interactions_df)
        ########################
        # Write your code here #
        # Prepare users_df and items_df
        # For users_df you need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations).
        # Note that for users who were not in the hotel before (which is true for most users)
        # there will be no features in self.users_df. For such users you can initialize their features
        # with all zeros (for instance with fillna(0)), but you can also try to use average feature
        # values from self.users_df (this way you would trear a new user as an average user).
        # For items you have to apply the prepare_items_df method to items_df.
        
        # Score the items
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            
            ########################
            user_df = pd.DataFrame([user],columns=users_df.columns)
            input_df = pd.merge(user_df,items_df,how="cross")
            ########################
            # Write your code here #
            # Create a Carthesian product of users from users_df and items from items_df

            ########################
            scores = self.model.predict(input_df.loc[:, self.user_features + item_features].values)
            ########################
            # Write your code here #
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            
            ########################
            chosen_ids = np.argsort(-scores)[:n_recommendations]
            ########################
            # Write your code here #
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
#             chosen_ids = None
            
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [7]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [8]:
# Fit method
cb_user_item_recommender = LinearRegressionCBUIRecommender()
# cb_user_item_recommender = SVRCBUIRecommender()
# cb_user_item_recommender = RandomForestCBUIRecommender()
# cb_user_item_recommender = XGBoostCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [9]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), items_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,56,1.016602,OffSeason,[2-3],Nonref,[0-160],[3-4],True
1,1,22,1.000000,OffSeason,[2-3],Standard,[0-160],[3-4],True
2,1,60,0.992188,OffSeason,[2-3],Nonref,[0-160],[2-2],True
3,1,93,0.981445,OffSeason,[2-3],Nonref,[160-260],[3-4],True
4,1,33,0.975586,OffSeason,[2-3],Standard,[0-160],[2-2],True
5,1,24,0.965820,OffSeason,[2-3],Standard,[160-260],[3-4],True
6,1,151,0.956055,OffSeason,[2-3],Nonref,[160-260],[2-2],True
7,1,35,0.941406,OffSeason,[2-3],Standard,[160-260],[2-2],True
8,1,129,0.937500,OffSeason,[2-3],Nonref,[0-160],[3-4],False
9,1,1019,0.929688,HighSeason,[2-3],Nonref,[0-160],[3-4],True


# Tuning method

In [10]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

! pip install hyperopt

In [11]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [82]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|███████████████████████████| 10/10 [12:18<00:00, 73.87s/trial, best loss: -0.18158783783783783]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.037288,0.102034,0.144407,0.187458,0.037288,0.073079,0.090198,0.104212


Best parameters:
{'n_neg_per_pos': 7.0}


In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  1%|▏                    | 1/100 [40:25<66:41:50, 2425.36s/trial, best loss: -0.001266891891891892]

In [83]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|███████████████████████| 100/100 [2:41:31<00:00, 96.92s/trial, best loss: -0.04391891891891892]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.000339,0.005085,0.010169,0.018983,0.000339,0.003023,0.005079,0.00787


Best parameters:
{'max_depth': 5.0, 'min_samples_split': 11.0, 'n_estimators': 99.0, 'n_neg_per_pos': 5.0}


In [107]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|███████████████████████| 300/300 [5:12:29<00:00, 62.50s/trial, best loss: -0.06714527027027027]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.008136,0.030508,0.043051,0.058644,0.008136,0.020787,0.025876,0.030803


Best parameters:
{'learning_rate': 0.057696366427124925, 'max_depth': 2.0, 'min_samples_split': 23.0, 'n_estimators': 112.0, 'n_neg_per_pos': 1.0}


# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [12]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(
    **{'n_neg_per_pos': 6})

linear_cbui_tts_results = [['Recommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,Recommender,0.037288,0.102034,0.144407,0.196271,0.037288,0.073079,0.090198,0.106975


In [13]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.035932,0.092542,0.132203,0.188475,0.035932,0.068587,0.084985,0.103051


In [14]:
tts_results = pd.concat([linear_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,Recommender,0.037288,0.102034,0.144407,0.196271,0.037288,0.073079,0.090198,0.106975
1,AmazonRecommender,0.035932,0.092542,0.132203,0.188475,0.035932,0.068587,0.084985,0.103051
